In [1]:
import string
import numpy as np

In [2]:
#import feature frames

import pandas as pd

x = 'Wikipedia'

wikipedia_test_data = pd.read_pickle('pickled-dataframes/'+x+'_test_data')
wikipedia_training_data = pd.read_pickle('pickled-dataframes/'+x+'_training_data')
wikipedia_test_data.name = x
wikipedia_training_data.name = x

x = 'News'

news_test_data = pd.read_pickle('pickled-dataframes/'+x+'_test_data')
news_training_data = pd.read_pickle('pickled-dataframes/'+x+'_training_data')
news_test_data.name = 'News'
news_training_data.name = 'News'

x = 'WikiNews'

wiki_test_data = pd.read_pickle('pickled-dataframes/'+x+'_test_data')
wiki_training_data = pd.read_pickle('pickled-dataframes/'+x+'_training_data')
wiki_test_data.name = x
wiki_training_data.name = x

FileNotFoundError: [Errno 2] No such file or directory: 'pickled-dataframes/Wikipedia_test_data'

In [40]:
lexicon = pd.read_table('lexicon', delim_whitespace=True, names=('phrase', 'score'))

In [41]:
lexicon['phrase'] = lexicon['phrase'].apply(lambda x: str(x).lower())

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler



first_fixation = Pipeline([
                ('selector', NumberSelector(key='IA_FIRST_FIXATION_DURATION')),
                ('standard', StandardScaler())
            ])

words =  Pipeline([
                ('selector', TextSelector(key='phrase')),
                ('vect', CountVectorizer())
            ])

word_length =  Pipeline([
                ('selector', NumberSelector(key='length')),
                ('standard', StandardScaler())
            ])

dep_num =  Pipeline([
                ('selector', NumberSelector(key='dep num')),
                ('standard', StandardScaler())
            ])


tag =  Pipeline([
                ('selector', TextSelector(key='pos')),
                ('vect', CountVectorizer())
            ])

synonyms =  Pipeline([
                ('selector', NumberSelector(key='synonyms')),
                ('standard', StandardScaler())
            ])

hypernyms =  Pipeline([
                ('selector', NumberSelector(key='hypernyms')),
                ('standard', StandardScaler())
            ])

hyponyms =  Pipeline([
                ('selector', NumberSelector(key='hyponyms')),
                ('standard', StandardScaler())
            ])

syllables =  Pipeline([
                ('selector', NumberSelector(key='syllables')),
                ('standard', StandardScaler())
            ])

simple_wiki =  Pipeline([
                ('selector', NumberSelector(key='simple_wiki')),
                ('standard', StandardScaler())
            ])

ogden =  Pipeline([
                ('selector', NumberSelector(key='ogden')),
                ('standard', StandardScaler())
            ])
origin =  Pipeline([
                ('selector', TextSelector(key='origin')),
                ('vect', CountVectorizer())
            ])

frequency =  Pipeline([
                ('selector', NumberSelector(key='google frequency')),
                ('standard', StandardScaler())
            ])

subimdb =  Pipeline([
                ('selector', NumberSelector(key='sub_imdb')),
                ('standard', StandardScaler())
            ])

n_gram_freq =  Pipeline([
                ('selector', NumberSelector(key='ngram freq')),
                ('standard', StandardScaler())
            ])

cald =  Pipeline([
                ('selector', NumberSelector(key='cald')),
                ('standard', StandardScaler())
            ])
aoa =  Pipeline([
                ('selector', NumberSelector(key='aoa')),
                ('standard', StandardScaler())
            ])
conc =  Pipeline([
                ('selector', NumberSelector(key='conc')),
                ('standard', StandardScaler())
            ])
fam =  Pipeline([
                ('selector', NumberSelector(key='fam')),
                ('standard', StandardScaler())
            ])
img =  Pipeline([
                ('selector', NumberSelector(key='img')),
                ('standard', StandardScaler())
            ])
phon =  Pipeline([
                ('selector', NumberSelector(key='phon')),
                ('standard', StandardScaler())
            ])

score =  Pipeline([
                ('selector', NumberSelector(key='score')),
                ('standard', StandardScaler())
            ])

In [8]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

global feats
feats = FeatureUnion([#('ff',first_fixation),
                      ('words', words),
                      ('word_length', word_length),
                      ('Tag', tag),
                      ('dep_num',dep_num),
                      ('hypernyms',hypernyms),
                      ('hyponyms',hyponyms),
                      ('synonyms', synonyms),
                      ('Syllables', syllables),
                      ('ogden',ogden),
                      ('simple_wiki', simple_wiki),
                      #('origin', origin),
                      ('freq',frequency),
                      ('subimdb', subimdb),
                      #('n_gram_freq',n_gram_freq),
                      ('cald', cald),
                      ('aoa', aoa),
                      ('conc', conc),
                      ('fam', fam),
                      ('img', img),
                      ('phon', phon),
                        ('score', score)
                     ])
feats

FeatureUnion(transformer_list=[('words',
                                Pipeline(steps=[('selector',
                                                 TextSelector(key='phrase')),
                                                ('vect', CountVectorizer())])),
                               ('word_length',
                                Pipeline(steps=[('selector',
                                                 NumberSelector(key='length')),
                                                ('standard',
                                                 StandardScaler())])),
                               ('Tag',
                                Pipeline(steps=[('selector',
                                                 TextSelector(key='pos')),
                                                ('vect', CountVectorizer())])),
                               ('dep_num',
                                Pipeline(steps=[(...
                                                 NumberSelector(key='

In [82]:
frames = [news_training_data,wikipedia_training_data,wiki_training_data]
total_training = pd.concat(frames)

frames =[wikipedia_test_data, wiki_test_data,news_test_data]
total_test = pd.concat(frames)

In [83]:
total_training = pd.merge(total_training, lexicon, on='phrase', how='left')
total_training.fillna(0.0, inplace=True)

In [84]:
total_test = pd.merge(total_test, lexicon, on='phrase', how='left')
total_test.fillna(0.0, inplace=True)

In [85]:
training_data = total_training
train_targets = training_data['complex_binary'].values
        
feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(training_data)
        
model = AdaBoostClassifier(n_estimators=5000, random_state=67)
pipeline = Pipeline([
    ('features',feats),
    ('classifier', model),
    ])

pipeline.fit(training_data, train_targets)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('words', Pipeline(memory=None,
     steps=[('selector', TextSelector(key='phrase')), ('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='conte...m='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=5000, random_state=67))])

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import scipy.stats as stats


global model_stats
model_stats= pd.DataFrame(columns=['Data','Classifier','Precision','Recall','F-Score'])

def apply_algorithm(array):
 


    
        i=0
        for x in array:
            
            test_data = x
            test_targets =  test_data['complex_binary'].values
            
      

            test_predictions = pipeline.predict(test_data)
                
            accuracy = accuracy_score(test_targets, test_predictions)
            precision = precision_score(test_targets, test_predictions)
            recall = recall_score(test_targets, test_predictions)
            F_Score = f1_score(test_targets, test_predictions)
               

            model_stats.loc[len(model_stats)] = [i,(str(model))[:60] ,precision, recall, F_Score]
            #baseline_accuracies(test_targets)


In [87]:
apply_algorithm([total_test]) #with lexicon
model_stats

,Data,Classifier,Precision,Recall,F-Score
0,0,"AdaBoostClassifier(algorithm='SAMME.R', base_e...",0.826087,0.838852,0.832421


In [80]:
apply_algorithm([total_test]) #without lexicon
model_stats

,Data,Classifier,Precision,Recall,F-Score
0,0,"AdaBoostClassifier(algorithm='SAMME.R', base_e...",0.823784,0.84106,0.832332
